# Assignment-1: Computational Genomics
## Task 1.2

In [ ]:
pip install scanpy

In [ ]:
import subprocess
subprocess.run('conda install -c conda-forge r-base', shell=True)

In [ ]:
!pip install rpy2

In [ ]:
!pip install anndata2ri

In [ ]:
import scanpy as sc 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import logging
import seaborn as sns
import anndata2ri
import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()


In [ ]:
matrix = sc.read('/kaggle/input/dataset.h5ad')
matrix

What does this output mean? 
- n_obs refers to the number of cells. Hence, here we have 10,727 cells. 
- n_vars refers to the number of genes. This matrix contains details about 12,303 genes. 

Inspecting the AnnData object. 

In [ ]:
matrix.var_names

In [ ]:
matrix.obs_names

Adding the gene IDs provided.

In [ ]:
gene_IDs = pd.read_csv('/kaggle/input/gene_names.csv')
gene_IDs

In [ ]:
matrix.var_names = gene_IDs['0'].values
matrix.var_names

### Quality Control
Finding the mitochondrial, ribosomal and hemoglobin genes. 

In [ ]:
# mitochondrial genes
matrix.var["mt"] = matrix.var_names.str.startswith("MT-")
# ribosomal genes
matrix.var["ribo"] = matrix.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes
matrix.var["hb"] = matrix.var_names.str.contains("^HB[^(P)]")
sc.pp.calculate_qc_metrics(
    matrix, qc_vars=["mt", "ribo", "hb"], inplace=True, log1p=True
)

The function above, adds the qc metrics to our AnnData object. The log1p=True returns the logarithm of the results (log(1+x)), which accounts for skewed distrubutions. 

In [ ]:
sc.pl.violin(
    matrix,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
    jitter=0.4,
    multi_panel=True,
)

It is clear from the above violin plots that there are no mitochondrial genes expressed. 

In [ ]:
sc.pl.scatter(matrix, "total_counts", "n_genes_by_counts")

To choose apt quality control thresholds, it may be helpful to draw histograms. 

In [ ]:
# Plot a histogram of total counts
plt.hist(matrix.obs['total_counts'], bins=100, color='skyblue', edgecolor='black')
plt.title('Total Counts per Cell')
plt.xlabel('Total Counts')
plt.ylabel('Number of Cells')
plt.show()

There seems to be a noise peak in the above plot below the value of 1400 total counts. 

In [ ]:
# Plot a histogram of number of genes per barcode
plt.hist(matrix.obs['n_genes_by_counts'], bins=100, color='skyblue', edgecolor='black')
plt.title('Number of Unique Genes Expressed Per Cell')
plt.xlabel('Number of Unique Genes')
plt.ylabel('Number of Cells')
plt.show()

In this graph there is no apparent noise peak on the lower end. 

In [ ]:
TotalCounts = matrix.obs['total_counts'].values
sorted_counts = np.sort(TotalCounts)[::-1]
ranks = np.arange(1, len(sorted_counts) + 1)
plt.figure(figsize=(8, 6))
plt.plot(ranks, sorted_counts, marker='o', linestyle='none', color='blue', markersize=1)
plt.yscale('log')
plt.xlabel('Rank of Cells (High to Low)')
ax = plt.gca()
ax.yaxis.set_major_locator(ticker.LogLocator(base=10.0, subs='auto'))
ax.yaxis.set_minor_locator(ticker.LogLocator(base=10.0, subs='auto', numticks=10))
ax.yaxis.set_major_formatter(ticker.LogFormatter(base=10.0))
plt.ylabel('Total Counts (log scale)')
plt.title('Count Depth Distribution (Log-Log Plot)')
plt.grid(True, which='both', linestyle='--', linewidth=0.7)
plt.show()

In the above graph, the "Elbow" is around 1600 total counts.

### Doublet Detection
Making use of Scrublet - the nearest neighbour based doublet detection algorithm.

In [ ]:
sc.pp.scrublet(matrix)
np.sum(matrix.obs['predicted_doublet'])

Hence, 161 doublets have been detected.

### Normalization


In [ ]:
#Saving the raw counts
matrix.layers["counts"] = matrix.X.copy()
# Normalizing to median total counts and transforming it to log(1+x)
sc.pp.normalize_total(matrix)
sc.pp.log1p(matrix)

### Feature Selection
This step is to reduce the dimensions of the matrix such that only the most informative genes are retained. Scanpy automatically annotates the highly variable genes using variance to mean ratios. Usually, erring towards the removal of a larger number of highly variable genes often produces the best results. Hence, here we choose the top 2000 most variable genes.   

In [ ]:

%load_ext rpy2.ipython

In [ ]:
%%R
if (!requireNamespace("BiocManager", quietly = TRUE)) {
    install.packages("BiocManager", quiet = TRUE, repos = "http://cran.us.r-project.org")
}

#installing the required packages
BiocManager::install("S4Vectors", ask = FALSE, update = FALSE, quiet = TRUE)
BiocManager::install("scry", ask = FALSE, update = FALSE, quiet = TRUE)


In [ ]:
%%R
library(scry)

sce = devianceFeatureSelection(matrix, assay="X")

binomial_deviance = rowData(sce)$binomial_deviance

idx = order(binomial_deviance, decreasing = TRUE)[1:2000]
mask = rep(FALSE, length(binomial_deviance))
mask[idx] = TRUE


In [ ]:
mask = ro.globalenv['mask']
binomial_deviance = ro.globalenv['binomial_deviance']

In [ ]:
# Back to Python
# Update the AnnData object with highly deviant genes
matrix.var["highly_deviant"] = mask
matrix.var["binomial_deviance"] = binomial_deviance

In [ ]:
matrix

As visible, the 'highly_variable' and 'binomial_deviance' key has now been added to var. 

### Dimensionality Reduction
Performing Principal Component Analysis (PCA).

In [ ]:
matrix = matrix.copy()
sc.tl.pca(matrix)
sc.pl.pca_variance_ratio(matrix, n_pcs=50, log=True)


The "elbow" of the above curve is around the region of 10-11 principal components. Since there are no pronounced disadvantages of overestimating the principal components, we can fix the value at 50 principal components.

In [ ]:
sc.pp.pca(matrix, n_comps = 50)

### Constructing the Nearest Neighbours Graph


In [ ]:
sc.pp.neighbors(matrix)
sc.tl.umap(matrix)
sc.pl.umap(matrix, size = 2,)

### Visualizing the QC Metrics

In [ ]:
sc.pl.umap(
    matrix,
    color=["predicted_doublet", "doublet_score"],
    # increase horizontal space between panels
    wspace=0.5,
    size=3,
)

In [ ]:
sc.pl.umap(
    matrix,
    color=["log1p_total_counts", "log1p_n_genes_by_counts"],
    wspace=0.5,
    ncols=2,
)

In [ ]:
sc.pl.umap(
    matrix,
    color=["log1p_total_counts", "log1p_n_genes_by_counts"],
    wspace=0.5,
    ncols=2,
)

Let us now find the highly variable genes for clustering.

In [ ]:
#5000
sc.pp.highly_variable_genes(matrix, n_top_genes=5000)
matrix_5000 = matrix[:, matrix.var["highly_variable"]]
sc.pp.neighbors(matrix_5000)
sc.tl.umap(matrix_5000)
sc.pl.umap(matrix_5000, color = ['total_counts'], size = 2,)


In [ ]:
matrix_5000

In [ ]:
matrix = matrix_5000

# Clustering
Making use of leiden clustering. Let us observe how the clusters form at multiple different resolutions. 

In [ ]:
for res in [0.05, 0.1, 0.20, 0.25, 1.0, 2.0]:
    sc.tl.leiden(
        matrix, key_added=f"leiden_res_{res:4.2f}", resolution=res, flavor="igraph"
    )

In [ ]:
sc.pl.umap(
    matrix,
    color=["leiden_res_0.05", "leiden_res_0.10", "leiden_res_0.20", "leiden_res_0.25", "leiden_res_1.00", "leiden_res_2.00" ],
    legend_loc="on data",
)

All clusters above 0.25 seem somewhat overclustered. Let us work further with the 0.25 resolution Leiden clusters.

In [ ]:
#Obtaining Output Data
leiden_out = matrix.obs['leiden_res_0.20']
leiden_df = pd.DataFrame(leiden_out)
leiden_df.to_csv('leiden-0.20-dev.csv', index=True)


In [ ]:
# Obtain cluster-specific differentially expressed genes
sc.tl.rank_genes_groups(matrix, groupby="leiden_res_0.20", method="wilcoxon")
sc.pl.rank_genes_groups_dotplot(
    matrix, groupby="leiden_res_0.20",standard_scale="var", n_genes=5
)